最近想了想怎么给我的博客科密码学的部分定位,本博客的定位为:
1. 本博客面向于对密码学,数学,算法感兴趣的读者
2. 读者不需要有大学一年级以上数学基础,或者编程基础,但是有自学能力.即对于文章提到的一些名词和函数(程序)可以通过查阅资料学会.
3. 本博客不会讲教科书密码学,会考虑实际问题
4. 不会照搬教科书和可直接通过简单的网络搜索就能得到的资料,会有原创内容


## 1:Mono-alphabetic Substitution Cipher
### 1.1:**加密原理**
**Mono-alphabetic Substitution Cipher**这是一古代密码,一种已经被破解的密码,本文会介绍**如何一键破解改密码**.该密码通常是大多数大学本科密码学教材的第一个密码,也是大多数完全不懂密码学的人都有印象的密码.

替换密码一句话概括即,重新定义"字母表",按照新的字母表替换明文中的每一个字母形成密文,这个字母表也就是所谓的**密钥(key)**.

正如所有的加密算法一样,抽象的模型都是在明文和密文之间建立一个映射$E$,使得:
$$E(PlainText)=CipherText$$

在**Mono-alphabetic Substitution Cipher**中用到的一种映射叫做**排列**(permutation).

具体来讲,**排列**是一个代数学概念,就是一个映射$p$:
$p = 
 \begin{pmatrix}
  1 & 2 & \cdots & n \\
  p(1) & p(2) & \cdots & p(n) \\
 \end{pmatrix}$
即,映射p把1变成p(1),把2变成p(2).....把n变成p(n),其中集合$\{p(1),p(2),...,p(n)\}$中的每一个元素都不一样,并且$\{p(1),p(2),...,p(n)\}$=$\{1,2,...n\}$.(注意集合是不考虑顺序的)
**例1**:
在这里如果给字符串k='abcd'进行编号对应'1234',并定义排列q
$q = 
 \begin{pmatrix}
  1 & 2 & 3 & 4\\
  4 & 3 & 2 & 1 \\
 \end{pmatrix}$
 即q(1)=4,q(2)=3,q(3)=2,q(4)=1
 于是$q\circ k=$'dcba' (这里$\circ$的意思是替换p作用于k)
 
 注:咱们把保持顺序原封不动的排列称之为**单位排列**,用符号$e$表示.比如本例当中单位排列则为:
 $e = 
 \begin{pmatrix}
  1 & 2 & 3 & 4\\
  1 & 2 & 3 & 4 \\
 \end{pmatrix}$

$\Box$
       
 这个排列并不是加密映射$E$,但是构成加密映射的一部分.Mono-alphabetic Substitution Cipher加密映射的关键在于字母表之间定义的映射.
 
 假设$k_0$是原字母表(不一定等于'abc....'因为特殊字符也有可能编入字母表进行替换),定义一种排序$p$,$k_p=p(k_0)$,定义集合(此处把字母表当成一个集合)$k_0$到$k$之间的**双射**(bijiective),$E$为加密映射:
 $$E_p(k_0[i])=k_p[i]$$
 
 其中$k_0[i]$,$k[i]$表示字母表中第i个元素.

 
 
 **例2**:
 若原字母表为$k_0$='abcd',排序为**例一**中的排序q,那么由加密映射表示出来则是:
 $$E_q(a)=d,E_q(b)=c,E_q(c)=b,E_q(d)=a$$
 
 令明文pliantext='aaabbdca',则经过加密以后的密文$Ciphertext=E_q(pliantext)$='dddccabd'
 
 $\Box$
 
注意,这里$E_q(pliantext)$的意思是这个映射作用于字符串的每一个单独的字母,然后结果变成一个新的字符串.

其次注意到**例二**中的映射不是plaintext里面字母打乱顺序,打乱顺序每一个字母的数量(**频数**)是不会改变的.如果是打乱顺序,那么密文中'a'的个数依旧是4,然而实际上频数是1.实际上频数因为替换发生了转移.不难发现,其实**例二**中的'a'的频数变成了'd'的频数,而'd'的频数变成了'a'的频数.如果把频数除以字符串中包含的字母总数,那么这可以理解为:

>**经过加密映射(替换)以后plaintext中字母的频率按照排列q的方式重新排序了**

理解这句话是破解类似的替换加密的关键!

**例3**:

通过统计**例二**中明文和密文每个字母的频率并分别建立一个**字典**(dictionary),如下:

$$d_0=\{a:\frac{4}{7},b:\frac{2}{7},c:\frac{1}{7},d:\frac{1}{7}\}$$

$$d_q=\{a:\frac{1}{7},b:\frac{1}{7},c:\frac{2}{7},d:\frac{4}{7}\}$$

其中$d_0$是plaintext的字典,$d_q$是密文的字典.

不难发现,$d_0$中对应字母的频率与$d_q$对应字母的频率之间能建立一个映射,而这个映射恰好是排列$q$.

$\Box$

以上三个例子是一个简化版的替换密码的讲解,理解这三个例子对理解替换密码的原理以及破解替换密钥会起到至关重要的作用.接下来的例子将真实一个真正的,Mono-alphabetic Substitution Cipher.

**例子4**:
```
PlainText='''
The main classical cipher types are transposition ciphers, which rearrange the order of letters in a message (e.g., 'hello world' becomes 'ehlol owrdl' in a trivially simple rearrangement scheme), and substitution ciphers, which systematically replace letters or groups of letters with other letters or groups of letters (e.g., 'fly at once' becomes 'gmz bu podf' by replacing each letter with the one following it in the Latin alphabet). Simple versions of either have never offered much confidentiality from enterprising opponents. An early substitution cipher was the Caesar cipher, in which each letter in the plaintext was replaced by a letter some fixed number of positions further down the alphabet. Suetonius reports that Julius Caesar used it with a shift of three to communicate with his generals. Atbash is an example of an early Hebrew cipher. The earliest known use of cryptography is some carved ciphertext on stone in Egypt (ca 1900 BCE), but this may have been done for the amusement of literate observers rather than as a way of concealing information. 
'''
```

若密钥为:
```
rpflczmhbjetqd osiaynvkuwgx
```
那么相应的密文为:
```
nbzrdpj rlqpyyjlpqrljsbzarngszyrpazrnap ysoyjnjo rljsbzay,rubjlbrazpaap hzrnbzroaczaromrqznnzayrj rprdzyyphzr(z.h.,r'bzqqoruoaqc'rfzlodzyr'zbqoqrouacq'rj rprnajkjpqqgryjdsqzrazpaap hzdz nrylbzdz),rp cryvfynjnvnjo rljsbzay,rubjlbrygynzdpnjlpqqgrazsqplzrqznnzayroarhaovsyromrqznnzayrujnbronbzarqznnzayroarhaovsyromrqznnzayr(z.h.,r'mqgrpnro lz'rfzlodzyr'hdxrfvrsocm'rfgrazsqplj hrzplbrqznnzarujnbrnbzro zrmoqqouj hrjnrj rnbzrqpnj rpqsbpfzn).ryjdsqzrkzayjo yromrzjnbzarbpkzr zkzarommzazcrdvlbrlo mjcz njpqjngrmaodrz nzasajyj hrosso z ny.rp rzpaqgryvfynjnvnjo rljsbzarupyrnbzrlpzyparljsbza,rj rubjlbrzplbrqznnzarj rnbzrsqpj nzwnrupyrazsqplzcrfgrprqznnzaryodzrmjwzcr vdfzaromrsoyjnjo yrmvanbzarcou rnbzrpqsbpfzn.ryvzno jvyrazsoanyrnbpnrevqjvyrlpzyparvyzcrjnrujnbrprybjmnromrnbazzrnorloddv jlpnzrujnbrbjyrhz zapqy.rpnfpybrjyrp rzwpdsqzromrp rzpaqgrbzfazurljsbza.rnbzrzpaqjzynrt ou rvyzromrlagsnohapsbgrjyryodzrlpakzcrljsbzanzwnro ryno zrj rzhgsnr(lpr1900rflz),rfvnrnbjyrdpgrbpkzrfzz rco zrmoarnbzrpdvyzdz nromrqjnzapnzrofyzakzayrapnbzarnbp rpyrprupgromrlo lzpqj hrj moadpnjo .r
```

$\Box$
注意这里密钥key里面包含了空格,也就是说空格也算作字母表中参与排序.因为在实际情况中,加密者是一定会把空格参替换,否则很容易知道每一个单词的长度以及在哪里短句,这样破译者很有可能从英文的单一字母单词,比如:a,I下手,以及双字母单词:an am if or in on of go up he we等入手破解,所以做这种替换加密一定会把空格纳入替换,比如本次加密中第一句
'rlqpyyjlpqrljsbzarngszyrpazrnap'因为替换了空格所以多个单词之间没有任何断句.(教科书不会讲这个,因为那是textbook cryptography.本博客所有文章都会考虑实际情况下的问题)

## 2:**频率分析原理**
**频率分析**是针对以上替换密码的一种有效的攻击方法.

首先有一个问题,既然是古代密码,为什么不可以**暴力破解**(brute force)?以现在计算机的运算能力应该有办法暴力破解这种密码不是吗?
### 2.1:暴力破解?
现在的计算机运算能力可以暴力破解,但是效率不高,因为暴力破解首先要想到的是**key space**的大小.即所有可能的密钥有多少种?

现在来看如何计算Mono-alphabetic Substitution Cipher的key space 大小
**例5**:
计算这种加密方法的key space 等价于字母表$k_0$一共有多少种排列.以**例4**为例,$k_0$的长度为27,于是再简化则是一个数学问题:

$\{1,2,...,27\}$一共有多少种排列?

答案是:27! (注意这里'!'是阶乘,即$27\times 26...\times1$)

$\Box$

这个结果大约等于$2^{93}$即相当于93bits的密钥强度,咱们知道20世纪末期的对称加密标准方法DES的密钥也不过64 bits,所以从key space的角度来讲,这并不是简单就能暴力破解的密码.但是为什么咱们说这是一种过时的密码?

### 2.2:密码攻击手段分类

针对于密码学的攻击手段大致分为三类:
1. **只知道密文的攻击(CipherText-only attack)**
这种攻击要求攻击者只知道密文,在这个情况下要破解出明文.这是理论上最难的一种攻击,如果攻击者能够对任意的此方法加密的密文展开CipherText-only attack,那么意味着,这种密码完全被破解,即完全不能使用.

2. **有一些明文信息或密码信息的攻击**
这一类攻击不仅知道密文,还掌握了有关于明文的信息,根据对明文不同程度的掌握分别有:
Known-plainText attack,Chosen-plaintext attack,Adaptive-chosen-plaintext attack等等,这里不做详细展开.

3. **特殊攻击**
比如Rubber-hose Cryptanalysis,意思是绑架持有密码的人并进行威胁或者严刑拷打等特殊都断迫使对方交出密钥.以及Purchase key attack 通过贿赂的手段使得对方出售密钥.

除开第三类特殊手段,本文的对象Mono-alphabetic Substitution Cipher无法抵御第一种攻击,即CipherText-only attack.也就是说,针对于该加密算法的攻击者在对明文几乎没有了解的情况下就能实施攻击,并破译密文.(当然知道密文是英文也算是一种信息,但是其实只要设计多种语言的频率分析,并借助**nature language indentification**(注:python包nlk有相应代码和函数)技术,则可以不需要知道明文是何种语言照样能破译)

### 2.3:什么是频率分析?

通常的教科书或者维基百科一定会讲频率分析,首先咱们来看教科书版(维基百科版)的频率分析是怎么回事.

首先一定会有这样的一个图表(histogram):
![3.png](#file:7b58d45b-c204-f9f2-144b-9fd46345cecd)
意思是:
**通过对密文种各个字母出现的频率进行统计,通过和标准频率进行比对来猜测密钥种包含的替换顺序.**

>回顾**例3**,该例子表明,这类替换一对一替换的加密算法,实际上频率发生的是相应的排列.那么只要找出这个排列,就找到了密钥,这可以破解密文.

上面这句话就是频率分析的核心思想.

而简单的频率分析(也就是**histogram频率**分析,指单个字母的频率统计来做频率分析)的想法是,拉丁语系的语言,不管是英文,西班牙语,希腊语,拉丁语字母出现的频率都不是均匀的,一定有高频字母(比如英文的:e)和低频单词(英文中的z).比如对大量的英文文献进行统计大致会得到这样'etaion.....'的字母频率由高到低的排列顺序.

因此如果统计密文中字母的频率并按照由高到低的排列顺序,并把这个排序的字母和英文的(或某种语言)'etaion.....'之间做一个映射,这样两个字母表之间的映射是不是就很接近密钥的排序呢?

并不是,因为效果非常差!之所以出现这样的现象是因为样本相对于几百万几千万词的大数据样本而言是非常小的,因此可能保持有个性,比如有些单词一段话没有一个e,或者很长都不断句,这就和大数据统计的结果不符合.

**例6**:

以**例4**中的密文为例,通过统计histogram频率,发现密文中字母由高到底的排列顺序为:$alpha_1$='rznapjyo bqlsdmvfughckwetxi',同时英文大数据中字母频率由高到低的排序为$alpha_2$=' etaoinshrdlcumwfgypbvkjxqz'如果在这两个字母表之间建立一一映射,并把这个映射作为密钥进行解密,结果如下:
```
tre uoih ldonnilod licrea tycen oae taohncsnitish licrean, grilr aeoaaohpe tre sabea sm dettean ih o uennope (e.p., 'redds gsadb' felsuen 'erdsd sgabd' ih o taivioddy niucde aeoaaohpeueht nlreue), ohb nwfntitwtish licrean, grilr nynteuotiloddy aecdole dettean sa paswcn sm dettean gitr strea dettean sa paswcn sm dettean (e.p., 'mdy ot shle' felsuen 'puq fw csbm' fy aecdolihp eolr dettea gitr tre she msddsgihp it ih tre dotih odcrofet). niucde veanishn sm eitrea rove hevea smmeaeb uwlr lshmibehtiodity masu ehteacainihp sccshehtn. oh eoady nwfntitwtish licrea gon tre loenoa licrea, ih grilr eolr dettea ih tre cdoihtekt gon aecdoleb fy o dettea nsue mikeb hwufea sm csnitishn mwatrea bsgh tre odcrofet. nwetshiwn aecsatn trot jwdiwn loenoa wneb it gitr o nrimt sm traee ts lsuuwhilote gitr rin peheaodn. otfonr in oh ekoucde sm oh eoady refaeg licrea. tre eoadient xhsgh wne sm layctspaocry in nsue loaveb licreatekt sh ntshe ih epyct (lo 1900 fle), fwt trin uoy rove feeh bshe msa tre ouwneueht sm diteaote sfneavean aotrea troh on o goy sm lshleodihp ihmsauotish. 
```

$\Box$
的确这样的破译结果要比密文好一点,至少断句的问题解决了,但是结果里面几乎没有一个正确的单词,如果从这里开始人工破译,那么过程是漫长的,而且需要破译者对多种语言有很深的理解.例如从高频单词,高频组合,单词结构,猜词等等入手相当麻烦,而本文要讲的方法是直接一键得到基本破译的译文,人工只需要稍微调整.

### 2.4:histogram频率分析的局限性

在大多数教科书和科普文章介绍中,所谓的频率分析其实特质单个字母的histogram频率分析.然而这种频率分析有很强的局限性,分析出来的效果很差.这也就是为什么咱们需要引入**bigram**的频率分析.

所谓bigram(bigram这个词汇来自于n-gram,当n=1即为histogram,n=2则为bigram)频率分析不分析单个字母的频率而是研究两个字母的组合.例如'aa','ap','b','z','pq',如果把空格归入字母表(不算' a',' b',' '单独拿出来),那么bigram一共有677种组合.
bigram频率分析就是试图通过这677种组合的频率统计数据来还原排列的顺序,得到最接近的密钥.

接下来咱们先把分析**bigramp频率**的方法套用在histogram上试图优化简单的频率分析,使得histogram频率分析的效果达到极限.并从结果来论证,为什么histogram无论如何优化也得不到足够好的结果.最后咱们再把这一套方法运用于bigram频率分析,看看bigram频率分析的结果能破译到什么程度.

>本文的方法是参考Thomas Jakobsen 1995年的文章**A Fast Method for Cryptoanalysis Substitution Cipher**过程如下:

此方法的原理是,知道样本小所以有个性,一定和大数据的结果有差异.那么可以先进行一次频率分析,并把这种结果下推测出的密钥作为初始密钥,破译的译文作为初始密文.然后建立一个评价函数,评价破译的好坏,接着对密钥对应的顺序进行迭代调整,试图把评价函数的分值做到最好,这样得到的译文理论上会远远好于单次频率分析的结果.

下面具体的使用上述方法把histogram频率分析做到极限,并论histogram的局限性.

**2.4.1:统计密文中字母的频率**
假设字母表中有空格,空格在字母表中的字母顺序为1,然后是a对应2,以此类推.

假设密文的加密密钥为k,k对应的排序映射为p.在这种排列顺序下定义的加密映射$E_p$作用下密文的histogram频率统计结果为:$H_p=\{h_{1}^p,h_{2}^p,...,h_{27}^p\}$
(其中下标表示对应的字母,比如1对应空格,2,对应a,3对应b,....,27对应z,也就是说下表永远代表字母顺序,而不是字符串中的顺序.上标p表示参数p,也就是字母表的排序p.)

注意这里上下标有这样的关系:
设$H_e=\{h_{1}^e,h_{2}^e,...,h_{27}^e\}$,回顾**例1**,e表示单位排序.所以这里表示的是明文中各个字母的频率.

而$H_p$与$H_e$中元素的关系是:
$$h_i^p=p(h_j^e)$$
下标的关系是:
$$j=p^{-1}(i)$$
注:$p^{-1}$表示p的逆映射

这里面$i$是密文字母表中表示字母顺序的下标,$j$则表示明文字母表中表示字母的顺序的下标.(**这里顺序特指字母顺序不是排列顺序,切记!**)

下面一个例子会表明为什么需要这样的符号以及代表的具体含义,这个例子能帮助读者理解符号.

**例7**:

这里以**例4**中为例,明文字母表为:
```
 abcdefghijklmnopqrstuvwxyz
```
密文字母表为:
```
rpflczmhbjetqd osiaynvkuwgx
```

这里密文中$h_1$空格的频率,实际上是明文中'n'的频率,于是有:
$$h_1^p=p(h_{15}^e)$$

再比如,密文中$h_2$代表的频率是'a'的频率,实际上在明文中是'r'的频率,于是有:
$$h_2^p=p(h_{19}^e)$$

再看下标的关系,这里注意到$p(19)=2$,即p('r')='a',于是:
$$19=p^{-1}(2)$$


$\Box$

接上文,密钥k对应的排序映射为$p$,而如果咱们破译的过程中猜测的排序为$q$,这时候咱们用$q$来对密文进行破译(即q的逆映射$q^{-1}$做出的字母表把密文对应到译文),得到的译文进行频率统计得到了:
$$H_{q^{-1}p}=\{h_{1}^{q^{-1}p},h_{2}^{q^{-1}p},...,h_{27}^{q^{-1}p}\}$$

结合上文以及**例7**,不难理解这里表达的是:
$$h_i^{q^{-1}p}=q^{-1}(h_{j}^p))=q^{-1}(p(h_{t}^e))$$
其中,i是q为排列译文中字母的顺序,j为密文中字母的顺序,t为明文中字母的顺序.

这里下标的关系为:
$$t=p^{-1}(j),j=q(i)$$
于是有
$$t=p^{-1}(q(i))$$


注意到,如果$p=q$,那么$q^{-1}p=pq^{-1}=e$.这也就意味着破译完全正确!因为在这个时候由$t=p^{-1}(q(i))$得到$t=i$.这意味着译文字母表的排序和明文字母表排序完全一致.也就是a对a,b对b的......这意味着破译完全正确!

用一个例子来解释一下
**例8**:
还是以**例4**中的排列为例:
```
 abcdefghijklmnopqrstuvwxyz
```
密文字母表为:
```
rpflczmhbjetqd osiaynvkuwgx
```

假设在破译过程中采用排列$q$进行破译,其中$q(a)=c$,即$q(2)=4$.这当然是一种错误的破译,假设在这个条件下有:
$$h_4^{q^{-1}p}=q^{-1}(h_{2}^p))=q^{-1}(p(h_{18}^e))$$
意思是,译文中c的频率其实是密文中的a的频率,而实际上是明文中r的频率.
$\Box$

最后定义:
$$r_q=q^{-1}p$$

这里注意理解**例7**,**例8**的内容,因为这对后面理解对评价函数模型中参数做**极大似然估计**有辅助作用.



**2.4.2:设计评价函数**

评价函数也就是评价**总体上**来讲,对于一段文字,其频率的顺序和一般英文的排列顺序有多么接近.这需要用到统计学的方法.

按照严格的数学表达,假设由**大数定律**(LLN:Law of Large Number)每个字母(在超大样本,大数据中)出现的频率收敛于$\mu_{i}$,并且每个字母出现再文中的随机变量是独立同分布的随机变量(i.i.d),并且都服从**正态分布**(Normal Distibution),设每个正态分布的均值为$\mu_{i}$,方差为$\sigma_{i}$.

注意这里的$\mu_{i}$表示大数据中第i个字母的均值,比如$\mu_{1}$就是大数据中空格的均值.(参考**例7**,下标不是字符里面的顺序是字母顺序,空格放在'a'前面)

假设这时候咱们开始猜加密用的排序参数$p$,咱们的猜测为排序$q$,按照排序$q$破译的译文中咱们观察到的histogram频率统计结果是$H=\{h_{1}^{r_q},h_{2}^{r_q},...,h_{27}^{r_q}\}$.(注意上文中**例7**,**例8**提到的符号)



而且咱们知道一个大致的概率分布模型,也即是每一个随机变量相互独立的遵循正态分布.但是问题的关键在于,咱们知道任意$h_{i}^q$的均值在集合$\{\mu_i|i\in[1,27]\}$当中,但是不清楚均值$\mu_{i}$和观察到的数据具体的对应关系.为了解决这个问题,咱们利用一个统计学的技巧研究:**似然函数**(likelihood function )



$$L(q|H)=\prod_{i}\frac{1}{\sigma_{i}\sqrt{2\pi}}e^{\frac{-(h_{i}^{r_q}-\mu_{i})^{2}}{2{\sigma^2_{i}}}}$$

这个函数右边的表达式有两种认识,第一种如果把参数q当成固定的,即参数q固定样本是可以变动的情况下,为H时候的概率.这个时候是含有参数q的样本的概率密度函数.

反过来,当观测到的样本固定为H,参数q是变动的,这就是**似然函数**.在样本H固定时,参数p变化则对应到不同的似然函数值$L(q|H)$,自然也会对应到一个概率值$P_{q}(X=H)$.这里有这样一个道理,若:
$$L(q^{'}|H)>L(q^{''}|H)$$
则
$$P_{q'}(X=H)>P_{q''}(X=H)$$

咱们倾向于认为概率最大的时候对应的q,才是最合理的q.应该发生的概率越大,越容易被观察到,反过来如果咱们已经观察到H,那么倾向于认为是使得它的似然函数最大的情况下的q是最合理的.这就是**极大似然估计**(MLE=Maximum likelihood estimate).


**2.4.3:极大似然估计**

先对数化:
$$-r\sum_{i}\frac{(h_{i}^{r_q}-\mu_{i})^{2}}{2{\sigma^2_{i}}}$$
其中r是正的常数

即最为合理的排序p,就是让式子$f(q)=\sum_{i}\frac{(h_{i}^{r_q}-\mu_{i})^{2}}{2{\sigma^2_{i}}}$最小的q,因此评价q是否合理只需要看函数f(q)的大小即可,函数值越小q的选取越是合理.

最后在实际的计算中,咱们发现不用预先估计频率的方差,可以用下面的评价函数来代替似然函数导出的评价函数,并且计算更简单:
$$f(q)=\sum_{i}|h_{i}^{r_q}-\mu_{i}|$$

**例9**:
仍然以**例4**为例
对于评价函数$f$,当$q=p$的时候,即对明文进行评价,均值通过小说**MobyDick**与**UncleTom’sCabin**这两本小说作为大数据样本得到.
$$f(p)=16.23241099768974$$
当$q=e$的时候也就是对密文进行评价,得到
$$f(e)=97.4071661031994$$
当q取**例6**的简单单次频率分析的排序时(假设为$q_{0}$),得到
$$f(q_{0})=7.476673503720965$$
$\Box$

该例子发现,简单histogram频率分析的结果在评价函数下评分优于明文,即简单histogram频率分析的结果比明文更接近于大数据的频率.然而这样的结果完全没有可读性,离真正的破译还差得远.所以这就是histogram频率分析的局限性,如果采用histogram频率分析方法,根本不需要进行迭代优化,因为明文本身就和大数据的频率存在差异,histogram分析中即便参数q最优也无法得到满意的译文.

因此咱们需要考虑bigram频率分析.

### **2.5:Bigram频率分析**
**2.5.1:bigram频率分析的评价函数**

沿用2.4中histogram中一样的符号,假设密文的加密密钥为k,k对应的排序映射为p.在这种排列顺序下定义的加密映射$E_p$作用下密文的bigram频率统计结果为:$B_p=\{b_{11}^p,b_{12}^p,...,b_{ij}^p...b_{27,27}^p\}$

>注在实际编程的过程中,实际上没有考虑空格与字母的组合,只是把空格单独放进去,即26*26+1=677种组合而不是,27*27=729.此处讲的是729的情形.

接上文,密钥k对应的排序映射为$p$,而如果咱们破译的过程中猜测的排序为$q$,这时候咱们用$q$来对密文进行破译(即q的逆映射$q^{-1}$做出的字母表把密文对应到译文),得到的译文进行频率统计得到了:
$$B_{r_q}=\{b_{11}^{r_q},b_{12}^{r_q},...,b_{ij}^{r_q},..,b_{27,27}^{r_q}\}$$
其中$r_q=q^{-1}p$

这里的理解和histogram的表示是一样的道理,把bigram的两个字母的组合看成有序对,有序对的每一个部分单独适用于**例7**,**例8**当中的下标的关系.用一个例子说明:

**例10**:
假设破译过程中使用的排序为q,q(a)=c,q(b)=m
那么对于$b_{4,13}^{r_q}$而言其下标
$$4=r_q(18),13=r_q(8)$$
这就意味着:
$$b_{4,13}^{r_q}=r_q(b_{18,8}^e)$$

即译文中'cm'的频率实际上是密文中'ab'的频率,实际上是明文中'rh'的频率
$\Box$

同样假设bigram频率,由**大数定律**(LLN:Law of Large Number)每个字母组合(在超大样本,大数据中)出现的频率收敛于$\mu_{ij}$,并且每个字母出现再文中的随机变量是独立同分布的随机变量(i.i.d),并且都服从**正态分布**(Normal Distibution),设每个正态分布的均值为$\mu_{ij}$,方差为$\sigma_{ij}$.

然后对似然函数做相同的极大似然估计,得到评价函数:
$$f_B(q)=\sum_{i,j}|b_{ij}^{r_q}-\mu_{ij}|$$

为什么这样的评价函数比histogram下的评价函数更好,咱们用一个例子来说明

**例11**:

仍然以**例4**为例
对于评价函数$f$,当$q=p$的时候,即对明文进行评价,均值通过小说**MobyDick**与**UncleTom’sCabin**这两本小说作为大数据样本得到.
$$f_B(p)=52.907890170334404$$
当$q=e$的时候也就是对密文进行评价,得到
$$f(e)=161.250126255512$$
当q取**例6**的简单单次频率分析的排序时(假设为$q_{0}$),得到
$$f(q_{0})=87.76646150696772$$
$\Box$

把上面的例子和**例9**对比会发现bigram的评价函数显然更客观.简单的频率分析得到的译文和明文的结果有相当大的差距.

**2.5.2:bigram频率分析的迭代算法**

之所以需要迭代是因为一次的bigram分析是不够的,需要通过评价函数来迭代试图找到最优的参数q.从统计的角度而言,这个过程就是试图通过迭代算法对统计模型进行极大似然估计.

具体算法为:
具体算法为:
```
def bigramFreqAnalysis(Cipher,n):
    Freqkey=FreqOrderkey(Cipher) #这里的key是按照频率排序来的,并不是标准格式的key,所以需要转换
    key=convertKey(Freqkey)      #转换为标准格式的key
    Translation=MonoAlphabeticCipher(key,Cipher,'decrypt')
    Score=bigramMatchScore(Translation)
# 1.按照频率对Freqkey进行交换,优化破译结果
    for b in range(1,27):
        a=0
        while a+b<=26:
            Freqkey0=Swap(Freqkey,Freqkey[a],Freqkey[a+b]) #交换是按照频率顺序交换的,所以是交换Freqkey而不是标准key
            key0=convertKey(Freqkey0)
            Translation0=MonoAlphabeticCipher(key0,Cipher,'decrypt')
            Score0=bigramMatchScore(Translation0)
            a+=1
            if Score0<Score:
                Freqkey=Freqkey0
                Translation=Translation0
                Score=Score0
# 2.往往第一阶段完成以后结果还不是最好,所以接下来随机的进行替换,使得结果更好

    key=convertKey(Freqkey) #得到上一个阶段最后的密钥的标准格式
    
    for i in range(0,n):  # n是循环的次数
        RandChoiceA=randint(0,26) #得到1到27的随机整数
        RandChoiceB=randint(0,26)
        if RandChoiceA!=RandChoiceB:
            key0=Swap(key,key[RandChoiceA],key[RandChoiceB]) #随机交换密钥中两个字母的顺序
            Translation0=MonoAlphabeticCipher(key0,Cipher,'decrypt') # 解密
            Score0=bigramMatchScore(Translation0) # 评价
            if Score0<Score: #如果评价更好,得到新的密钥
                key=key0
                Translation=Translation0
                Score=Score0
                
    return Translation
```

1. 其中FreqOrderKey()函数表示简单的进行histogram频率统计,并按照频率由高到底排列顺序,然后把这个排序转换成标准的密钥格式.因为这样的排序是和' etaoi...'对应的,而不是和标准的字母表' abcd....'对应,因此需要转换.
2. MonoAlphabeticCipher(key,Cipher,'decrypt')意思是,按照密钥key对Cipher进行解密
3. bigramMatchScore()也就是评价函数$f_B$
4. Swap(key,key[a],key[a+b])意思是交换key中key[a]和key[a+b]对应的元素.


用一个例子来说明这个过程.
**例12**:

1. 以**例4**中的密文为例,首先进行简单的histogram频率分析,使用函数FreqOrderkey()
得到初始密钥
但是这个密钥不是标准格式的密钥,标准格式的密钥是密钥的字母表和标准值字母表,即' etaoi.....'对应,所以还需要一个用函数convertKey()把它转换成标准密钥以便于解密
```
Freqkey=FreqOrderkey(Cipher) 
key=convertKey(Freqkey)
```
结果:
```
key='rznjya pobsvlqmdcfhgwkuietx'
```
2. 使用这个密钥对密文进行破译,得到初始译文.这一步用到解密函数MonoAlphabeticCipher()
```
Translation=MonoAlphabeticCipher(key,Cipher,'decrypt')
Translation
```
结果:
```
'an cipdthyisdrp, s olgotatltahn cadrei ao s wetrhf hm encipdtany gp kracr lnato hm dusantebt sie ieduscef katr cadreitebt, scchifany th s mabef opotew; tre "lnato" wsp ge oanyue uetteio (tre whot chwwhn), dsaio hm uetteio, tiadueto hm uetteio, wabtlieo hm tre sghve, snf oh mhitr. tre ieceavei fecadreio tre tebt gp deimhiwany tre anveioe olgotatltahn.olgotatltahn cadreio csn ge chwdsief katr tisnodhoatahn cadreio. an s tisnodhoatahn cadrei, tre lnato hm tre dusantebt sie iesiisnyef an s fammeient snf lolsuup jlate chwdueb hifei, glt tre lnato trewoeuveo sie uemt lncrsnyef. gp chntisot, an s olgotatltahn cadrei, tre lnato hm tre dusantebt sie ietsanef an tre oswe oejlence an tre cadreitebt, glt tre lnato trewoeuveo sie suteief.treie sie s nlwgei hm fammeient tpdeo hm olgotatltahn cadrei. am tre cadrei hdeisteo hn oanyue uetteio, at ao teiwef s oawdue olgotatltahn cadrei; s cadrei trst hdeisteo hn usiyei yihldo hm uetteio ao teiwef dhupyisdrac. s whnhsudrsgetac cadrei loeo mabef olgotatltahn hvei tre entaie weoosye, kreieso s dhupsudrsgetac cadrei loeo s nlwgei hm olgotatltahno st fammeient dhoatahno an tre weoosye, kreie s lnat mihw tre dusantebt ao wsddef th hne hm oeveisu dhooagauataeo an tre cadreitebt snf vace veios. '
```
3. 对初始密文用bigram频率分析的评价函数,bigramMatchScore()进行评价得到初始得分:
```
Score=bigramMatchScore(Translation)
```
结果:
```
Score=114.70348811624602
```
4. 开始第一阶段的密钥交换.这一阶段的密钥交换的想法是,histogram频率(单个字母频率)接近的两个字母进行交换.交换是先从比较接近的,即频率排序Freqkey当中相邻的两个字母进行交换,然后字符间距为2的进行交换,间距为3进行交换,一直到间距为26的进行交换.这种搜索密钥的效率远远高于随机的搜索,而且一共只要搜索300+次.

变量b代表的是间距为多少的字母进行交换,比如当b=1的时候就是相邻的字母进行交换.
首先:
```
FreqKey0=Swap(FreqKey,FreqKey[0],FreqKey[1])
FreqKey0
```
得到:
```
FreqKey0='zrnjya pobsvlqmdcfhgwkuietx'
```
即'r'与'z'之间交换顺序
然后转化为标准密钥
```
key0=convertKey(FreqKey0)
key0
```
得到:
```
'zjwlsrcfoaiuvm ygtbpnqkdehx'
```
尝试使用这个密钥进行解密
```
Translation0=MonoAlphabeticCipher(key0,Cipher,'decrypt')
Translation0
```
得到一个有待评价的译文
```
'anecipdthyisdrp,eseolgotatltahnecadr ieaoesew trhfehme ncipdtanyegpekracrelnatoehmedusant btesi ei dusc fekatrecadr it bt,escchifanyethesemab feopot w;etr e"lnato"ewspeg eoanyu eu tt ioe(tr ewhotechwwhn),edsaioehmeu tt io,etiadu toehmeu tt io,ewabtli oehmetr esghv ,esnfeohemhitr.etr ei c av ief cadr ioetr et btegped imhiwanyetr eanv io eolgotatltahn.olgotatltahnecadr ioecsneg echwdsi fekatretisnodhoatahnecadr io.eanesetisnodhoatahnecadr i,etr elnatoehmetr edusant btesi ei siisny feanesefamm i ntesnfelolsuupejlat echwdu behif i,egltetr elnatoetr wo uv oesi eu mtelncrsny f.egpechntisot,eaneseolgotatltahnecadr i,etr elnatoehmetr edusant btesi ei tsan feanetr eosw eo jl nc eanetr ecadr it bt,egltetr elnatoetr wo uv oesi esut i f.tr i esi esenlwg iehmefamm i ntetpd oehmeolgotatltahnecadr i.eametr ecadr iehd ist oehneoanyu eu tt io,eateaoet iw feseoawdu eolgotatltahnecadr i;esecadr ietrstehd ist oehneusiy ieyihldoehmeu tt ioeaoet iw fedhupyisdrac.esewhnhsudrsg tacecadr ielo oemab feolgotatltahnehv ietr e ntai ew oosy ,ekr i soesedhupsudrsg tacecadr ielo oesenlwg iehmeolgotatltahnoestefamm i ntedhoatahnoeanetr ew oosy ,ekr i eselnatemihwetr edusant bteaoewsdd fethehn ehmeo v isuedhooagauata oeanetr ecadr it btesnfevac ev ios.e'
```
进行评价
```
Score0=bigramMatchScore(Translation0)
```
得到评分
```
Score0=123.99964329967796
```
和一开始的Score进行比较发现结果并没有变好,所以不采纳这个密钥

下一步开始交换
```
FreqKey0=Swap(FreqKey,FreqKey[1],FreqKey[2])
```
即频率第二的和第三的进行交换.....
一直到
交换
```
FreqKey0=Swap(FreqKey,FreqKey[3],FreqKey[4])
```
这时候译文的得分为:
```
Score0=114.55117931996695
```
进行比较发现比一开始的Score更低
于是采纳这样的密钥和密文
于是从下一次循环开始
```
FreqKey='rznyja pobsvlqmdcfhgwkuietx'
```

当b=1循环完成以后,开始间距为2的交换的循环
一致到间距为b=26的循环,这样第一阶段的循环就结束了
现在看一下结果
密钥:
```
key='rjwlszcfoaiuvm ygtbpnqkdehx'
```
译文:
```
ic gnyptofnaphy, a substitutioc giphen is a method ol ecgnypticf by whigh ucits ol praictevt ane nepraged with giphentevt, aggondicf to a lived system; the "ucits" may be sicfre rettens (the most gommoc), pains ol rettens, tniprets ol rettens, mivtunes ol the aboke, acd so lonth. the negeiken degiphens the tevt by penlonmicf the ickense substitutioc.substitutioc giphens gac be gompaned with tnacspositioc giphens. ic a tnacspositioc giphen, the ucits ol the praictevt ane neannacfed ic a dillenect acd usuarry xuite gomprev onden, but the ucits themserkes ane relt ucghacfed. by goctnast, ic a substitutioc giphen, the ucits ol the praictevt ane netaiced ic the same sexuecge ic the giphentevt, but the ucits themserkes ane artened.thene ane a cumben ol dillenect types ol substitutioc giphen. il the giphen openates oc sicfre rettens, it is tenmed a simpre substitutioc giphen; a giphen that openates oc ranfen fnoups ol rettens is tenmed poryfnaphig. a mocoarphabetig giphen uses lived substitutioc oken the ectine messafe, wheneas a poryarphabetig giphen uses a cumben ol substitutiocs at dillenect positiocs ic the messafe, whene a ucit lnom the praictevt is mapped to oce ol sekenar possibirities ic the giphentevt acd kige kensa. 
```
得分:
```
Score=82.97589096314215
```
这个时候的结果已经有很多人的词语可以辨认了,只是有一两个字母还不正确.但是这并不是意味着bigram分析的极限到了,因为如果咱们比较明文和当前译文的差距,可以发现还是有一点差距的,也就是说还有优化的空间.

6. 开始第二阶段的密钥搜索.这个时候搜索的想法就非常简单了,就是简单的随机的搜索,也就是任意交换两个密钥的顺序.这里有一个参数n,代表随机交换多少次.在进行了交换以后用评价函数的分数来决定交换后的密钥是否接受.

```
for i in range(0,n):  # n是循环的次数
        RandChoiceA=randint(0,26) #得到1到27的随机整数
        RandChoiceB=randint(0,26)
        if RandChoiceA!=RandChoiceB:
            key0=Swap(key,key[RandChoiceA],key[RandChoiceB]) #随机交换密钥中两个字母的顺序
            Translation0=MonoAlphabeticCipher(key0,Cipher,'decrypt') # 解密
            Score0=bigramMatchScore(Translation0) # 评价
            if Score0<Score: #如果评价更好,得到新的密钥
                key=key0
                Translation=Translation0
                Score=Score0

```
$\Box$

最后通过一个例子来看一下结果:
**例13**:
输入测试使用的明文
```
PlainText='''
In cryptography, a substitution cipher is a method of encrypting by which units of plaintext are replaced with ciphertext, according to a fixed system; the "units" may be single letters (the most common), pairs of letters, triplets of letters, mixtures of the above, and so forth. The receiver deciphers the text by performing the inverse substitution.

Substitution ciphers can be compared with transposition ciphers. In a transposition cipher, the units of the plaintext are rearranged in a different and usually quite complex order, but the units themselves are left unchanged. By contrast, in a substitution cipher, the units of the plaintext are retained in the same sequence in the ciphertext, but the units themselves are altered.

There are a number of different types of substitution cipher. If the cipher operates on single letters, it is termed a simple substitution cipher; a cipher that operates on larger groups of letters is termed polygraphic. A monoalphabetic cipher uses fixed substitution over the entire message, whereas a polyalphabetic cipher uses a number of substitutions at different positions in the message, where a unit from the plaintext is mapped to one of several possibilities in the ciphertext and vice versa. 
'''
```
选择密钥进行加密
```
key='rpflczmhbjetqd osiaynvkuwgx'
Cipher=MonoAlphabeticCipher(key,PlainText,'encrypt')
```
下面开始测试评价解密结果
```
破译结果:
in cryptography, a substitution cipher is a method of encrypting by which units of plaintekt are replaced with ciphertekt, according to a fiked system; the "units" may be single letters (the most common), pairs of letters, triplets of letters, miktures of the above, and so forth. the receiver deciphers the tekt by performing the inverse substitution.substitution ciphers can be compared with transposition ciphers. in a transposition cipher, the units of the plaintekt are rearranged in a different and usually quite complek order, but the units themselves are left unchanged. by contrast, in a substitution cipher, the units of the plaintekt are retained in the same sequence in the ciphertekt, but the units themselves are altered.there are a number of different types of substitution cipher. if the cipher operates on single letters, it is termed a simple substitution cipher; a cipher that operates on larger groups of letters is termed polygraphic. a monoalphabetic cipher uses fiked substitution over the entire message, whereas a polyalphabetic cipher uses a number of substitutions at different positions in the message, where a unit from the plaintekt is mapped to one of several possibilities in the ciphertekt and vice versa. 
破译评价:
==============破译时间:2.628910207748413 seconds
==============明文得分:71.11314852297146 ponints
==============密文得分:173.5862204717452 ponints
==============histogram译文得分:114.70348811624602 ponints
==============bigram译文得分:71.09227060658752 ponints


```
$\Box$
## **3:总结**
1. histogram因为想法过于简单,不适合作为评价函数,因此histogram频率分析方法效率很低,破译结果差,不能进一步优化.所以在进行频率分析破解替换密钥的过程要舍弃这种分析方法.
2. bigram频率分析过程中,如果密文种包含了空格,那么要把空格和26个字母的所有组合全部纳入bigram的字典当中.
3. bigram频率分析的优化(也就是进行极大似然估计过程)的过程中是建立在histogram简单分析的基础上的,第一阶段的搜索也是按照频率进行搜索的.这样可以很大程度上减少搜索的步骤.
4. bigram频率分析优化过程中第一阶段搜索完成以后结果还不是最优,为了达到最优可进行随机搜索进行优化,在第一阶段的基础上进行随机搜索大致需要1000-2000次搜索.即整个Bigram分析算法大致需要1300-2300次密钥搜索.


